In [1]:
import requests
import csv
from datetime import datetime

keys, transactions = None, []
current_time = 1654016400
last_transactions = set()

while True:
    query = """
    {
        liquidationCalls(where: {timestamp_lte: """ + str(current_time) + """}, first: 1000, orderBy: timestamp, orderDirection: desc) {
            id
            pool {
                id
            }
            user {
                id
            }
            collateralReserve {
                symbol
            }
            collateralAmount
            principalReserve {
                symbol
            }
            principalAmount
            liquidator
            timestamp
        }
    }
    """
    response = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                                '',
                                json={'query': query})
    if response.status_code != 200:
        print("Problem reading from timestamp", current_time, ":", response.status_code)
        continue
    
    try:
        data = response.json()["data"]["liquidationCalls"]
    except Exception:
        print("Error at timestamp", current_time)
        continue

    if len(data) == 0:
        break
    
    if keys is None:
        keys = data[0].keys()

    index = 0
    try:
        while data[index]["id"] in last_transactions:
            index += 1
    except IndexError:
            current_time -= 1
            continue

    print(len(data) - index, "transactions found at timestamp", current_time)

    for transaction in data[index:]:
        transaction["user"] = transaction["user"]["id"]
        transaction["pool"] = transaction["pool"]["id"]
        transaction["collateralReserve"] = transaction["collateralReserve"]["symbol"]
        transaction["principalReserve"] = transaction["principalReserve"]["symbol"]
        transactions.append(transaction)

    current_time = int(data[-1]["timestamp"])
    index = -1
    last_transactions = set()
    while data[index]["timestamp"] == data[-1]["timestamp"]:
        last_transactions.add(data[index]["id"])
        index -= 1

with open('liquidationCall.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(transactions)
        

1000 transactions found at timestamp 1654016400
999 transactions found at timestamp 1653041393
999 transactions found at timestamp 1652380006
999 transactions found at timestamp 1652301389
999 transactions found at timestamp 1652093439
999 transactions found at timestamp 1646720624
999 transactions found at timestamp 1644072109
997 transactions found at timestamp 1642845050
999 transactions found at timestamp 1642457946
997 transactions found at timestamp 1638595897
998 transactions found at timestamp 1628394372
999 transactions found at timestamp 1624321948
998 transactions found at timestamp 1621848531
999 transactions found at timestamp 1621773580
999 transactions found at timestamp 1621606371
998 transactions found at timestamp 1621424389
522 transactions found at timestamp 1614299738


In [2]:
!wc -l liquidationCall.csv
# 15566

16501 liquidationCall.csv


In [3]:
from google.colab import files
files.download('liquidationCall.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>